In [102]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, DateType, DoubleType
from pyspark.sql import functions
import math
import plotly.offline as pyo
import plotly.graph_objs as go
from calendar import month
from pandas import to_datetime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col
import pandas as pd

In [88]:
spark = SparkSession \
    .builder \
    .appName("airlinedelay") \
    .getOrCreate()

context = spark.sparkContext

In [89]:
schema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("OP_CARRIER", StringType(), True),
    StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("CRS_DEP_TIME", DoubleType(), True),
    StructField("DEP_TIME", DoubleType(), True),
    StructField("DEP_DELAY", DoubleType(), True),
    StructField("TAXI_OUT", DoubleType(), True),
    StructField("WHEELS_OFF", DoubleType(), True),
    StructField("WHEELS_ON", DoubleType(), True),
    StructField("TAXI_IN", DoubleType(), True),
    StructField("CRS_ARR_TIME", DoubleType(), True),
    StructField("ARR_TIME", DoubleType(), True),
    StructField("ARR_DELAY", DoubleType(), True),
    StructField("CANCELLED", DoubleType(), True),
    StructField("CANCELLATION_CODE", StringType(), True),
    StructField("DIVERTED", DoubleType(), True),
    StructField("CRS_ELAPSED_TIME", DoubleType(), True),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), True),
    StructField("AIR_TIME", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
    StructField("CARRIER_DELAY", DoubleType(), True),
    StructField("WEATHER_DELAY", DoubleType(), True),
    StructField("NAS_DELAY", DoubleType(), True),
    StructField("SECURITY_DELAY", DoubleType(), True),
    StructField("LATE_AIRCRAFT_DELAY", DoubleType(), True)
])

df = spark.read.format("csv") \
    .option("sep", ",") \
    .option("header", True) \
    .schema(schema) \
    .load("*.csv")

df.createOrReplaceTempView("airlinedelay")

In [90]:
df.show()

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|2016-01-01|        DL|             1

TOTAL RECORDS

In [91]:
spark.sql("SELECT count(*) FROM airlinedelay").show()

+--------+
|count(1)|
+--------+
|18505725|
+--------+



 AVERAGE DEPARTURE DELAY PER MONTH IN ALL YEARS

In [106]:

averageDepartureDelayPerMonth = spark.sql("SELECT CASE EXTRACT(month from FL_DATE) WHEN 1 THEN 'January'  WHEN 2 THEN 'February' WHEN 3 THEN 'March' WHEN 4 THEN 'April' WHEN 5 THEN 'May' \
WHEN 6 THEN 'June' WHEN 7 THEN 'July' WHEN 8 THEN 'August' WHEN 9 THEN 'September' \
WHEN 10 THEN 'October' WHEN 11 THEN 'November' ELSE 'December' END AS Month, \
AVG(IF(DEP_DELAY >= 0 , DEP_DELAY, NULL)) AS AverageDepartureDelayPerMonth, AVG(IF(ARR_DELAY >= 0 , ARR_DELAY, NULL)) AS AverageArrivalDelayPerMonth FROM airlinedelay GROUP BY Month")
averageDepartureDelayPerMonth.show()


+---------+-----------------------------+---------------------------+
|    Month|AverageDepartureDelayPerMonth|AverageArrivalDelayPerMonth|
+---------+-----------------------------+---------------------------+
| February|           30.735431890196587|          33.71100564562122|
|  January|            32.18828878699574|          35.25426456636053|
|    March|           28.878667663204894|         30.986196824550824|
|      May|            31.56993455797027|          34.25158585502806|
|    April|            31.86281084247858|          34.09016641899735|
|     July|           37.292714589718095|          40.61094899363225|
|     June|            34.88871055929404|          38.14195843797705|
|  October|            27.35363224271801|           29.2535701323158|
|   August|            36.20854900123184|         38.999591241859285|
|September|           28.909569347281945|          31.14594036226316|
| November|            27.16239535928725|         30.007738214623405|
| December|         

In [105]:
dfpandas = averageDepartureDelayPerMonth.toPandas()
months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]
dfpandasaverageDepartureDelayPerMonth = dfpandas.sort_values('Month', key = lambda x : pd.Categorical(x, categories=months, ordered=True))

averageDepartureDelayPerMonthGraph = go.Scatter(
    y = dfpandasaverageDepartureDelayPerMonth['AverageDepartureDelayPerMonth'],
    x = dfpandasaverageDepartureDelayPerMonth['Month'],
    mode = 'lines+markers',
    name = 'Average Departure Delay',
    marker = dict(color='red'),
    text = dfpandasaverageDepartureDelayPerMonth['Month'])

averageArrivalDelayPerMonthGraph = go.Scatter(
    y = dfpandasaverageDepartureDelayPerMonth['AverageArrivalDelayPerMonth'],
    x = dfpandasaverageDepartureDelayPerMonth['Month'],
    mode = 'lines+markers',
    name = 'Average Arrival Delay',
    marker = dict(color='yellow'),
    text = dfpandasaverageDepartureDelayPerMonth['Month'])

data = [averageDepartureDelayPerMonthGraph, averageArrivalDelayPerMonthGraph]

layout = dict(title = 'Evolution Of AVERAGE DEPARTURE/ARRIVAL DELAY PER MONTH IN ALL YEARS',
              xaxis= dict(title= 'Month',ticklen= 5,zeroline= False,tickformat='d'),
              yaxis= dict(title= 'Average Delay (Minutes)'),
              height= 600,
              width= 900
             )
fig = dict(data = data, layout = layout)

pyo.iplot(fig)

AVERAGE DEPARTURE DELAY PER CARRIER FOR EACH YEAR

In [95]:
averageDelayPerCarrier = spark.sql("SELECT OP_CARRIER, AVG(IF(DEP_DELAY >= 0 , DEP_DELAY, NULL)) AS AverageDelayPerCarrier, EXTRACT(year from FL_DATE) AS Year FROM airlinedelay GROUP BY OP_CARRIER, YEAR ORDER BY AverageDelayPerCarrier DESC" )

averageDelayPerCarrier.show()

+----------+----------------------+----+
|OP_CARRIER|AverageDelayPerCarrier|Year|
+----------+----------------------+----+
|        EV|     56.60643408878703|2018|
|        EV|     54.49749016415683|2017|
|        9E|     52.29959991640044|2018|
|        OO|    49.596884774419706|2017|
|        OO|     49.00604120365471|2018|
|        F9|     47.33786048460303|2018|
|        B6|     45.86273824773883|2017|
|        EV|    45.038134348244974|2016|
|        YV|     43.88718991345109|2018|
|        B6|     43.72999099548286|2018|
|        F9|     43.09401987941991|2016|
|        NK|    42.209263382238206|2017|
|        YX|     41.46272772954086|2018|
|        NK|     40.66915626492054|2018|
|        UA|    39.863071110562636|2018|
|        OO|    39.496158502777874|2016|
|        G4|     38.84540625521069|2018|
|        F9|     38.60866406760782|2017|
|        NK|    37.337342546482176|2016|
|        VX|     36.26525064795499|2017|
+----------+----------------------+----+
only showing top

In [111]:
traces = []
changePerCarrierMap = dict()
years = []

for i in range(len(averageDelayPerCarrier.toPandas()['Year'])):
    if not averageDelayPerCarrier.toPandas()['Year'][i] in years:
        years.append(averageDelayPerCarrier.toPandas()['Year'][i])

years.sort()
        
for i in range(len(averageDelayPerCarrier.toPandas()['OP_CARRIER'])):
    if not averageDelayPerCarrier.toPandas()['OP_CARRIER'][i] in changePerCarrierMap:
        changePerCarrierMap[averageDelayPerCarrier.toPandas()['OP_CARRIER'][i]] = []
        
    changePerCarrierMap[averageDelayPerCarrier.toPandas()['OP_CARRIER'][i]].append(averageDelayPerCarrier.toPandas()['AverageDelayPerCarrier'][i])
    
for key in changePerCarrierMap:
    traces.append(go.Bar(
        y = changePerCarrierMap[key],
        x = years,
        name = key))
                  
layout = dict(title = 'AVERAGE DEPARTURE DELAY PER CARRIER FOR EACH YEAR',
              xaxis= dict(title= 'Year',zeroline= False,tickformat='d'),
              yaxis= dict(title= 'Flight Delay (Minutes)',tickformat='d'),
              height= 650,
              width= 900
             )

fig = dict(data = traces, layout = layout)

pyo.iplot(fig)

COMPANY WITH THE LARGEST AVERAGE DELAY

In [114]:
companyWithLargetstAverageDelayForALLYears = spark.sql("SELECT OP_CARRIER, AVG(IF(DEP_DELAY >= 0 , DEP_DELAY, NULL)) AS AverageDelayPerCarrier FROM airlinedelay GROUP BY OP_CARRIER ORDER BY AverageDelayPerCarrier DESC LIMIT 1")

companyWithLargetstAverageDelayForALLYears.show()

+----------+----------------------+
|OP_CARRIER|AverageDelayPerCarrier|
+----------+----------------------+
|        9E|     52.29959991640044|
+----------+----------------------+



In [115]:
averageDepartureDelayPerYearTrace = go.Pie(
    values = companyWithLargetstAverageDelayForALLYears.toPandas()['AverageDelayPerCarrier'],
    text = companyWithLargetstAverageDelayForALLYears.toPandas()['OP_CARRIER'])


data = [averageDepartureDelayPerYearTrace]

layout = dict(title = 'Evolution Of Average Delay Year-over-Year',
              xaxis= dict(title= 'Years',ticklen= 5,zeroline= False,tickformat='d'),
              yaxis= dict(title= 'Average Delay (Minutes)'),
              height= 600,
              width= 900
             )
fig = dict(data = data, layout = layout)

pyo.iplot(fig)

AVERAGE DEPARTURE DELAY PER DEPARTURE LOCATION IN ALL YEARS

In [113]:
averageDelayPerDepartureLocationInAllYears = spark.sql("SELECT ORIGIN, AVG(IF(DEP_DELAY >= 0 , DEP_DELAY, NULL)) AS AverageDelayPerOrigin FROM airlinedelay  \
GROUP BY ORIGIN ORDER BY AverageDelayPerOrigin DESC")

averageDelayPerDepartureLocationInAllYears.show(50);

+------+---------------------+
|ORIGIN|AverageDelayPerOrigin|
+------+---------------------+
|   ENV|                157.0|
|   PPG|              121.875|
|   SLN|    88.72018348623853|
|   LWB|    87.99009900990099|
|   SHD|    82.14367816091954|
|   CMX|    75.48494983277592|
|   MQT|    74.36679536679537|
|   LBL|    74.19811320754717|
|   MEI|     73.9291553133515|
|   PUB|    72.18181818181819|
|   DVL|    71.73023255813953|
|   RDD|    68.96987253765933|
|   EGE|     68.9584552369807|
|   BGM|     68.3667711598746|
|   MBS|    68.17650834403081|
|   CKB|    67.97377049180328|
|   LCH|    67.80292942743009|
|   RHI|    67.14951456310679|
|   OTH|    66.96222222222222|
|   PIB|    66.46075085324232|
|   CGI|    66.34939759036145|
|   ITH|    66.28314917127072|
|   SAF|     65.7936210131332|
|   ISN|    65.73929471032746|
|   JMS|    64.92836257309942|
|   LRD|    64.62005856515373|
|   SCE|    64.50854700854701|
|   UIN|    64.31189710610933|
|   CLL|    64.19959473150962|
|   MHK|

AVERAGE DEPARTURE DELAY PER DEPARTURE LOCATION EACH YEAR

In [112]:
averageDelayPerDepartureLocationEachYear = spark.sql("SELECT ORIGIN, AVG(IF(DEP_DELAY >= 0 , DEP_DELAY, NULL)) AS AverageDelayPerCarrier, EXTRACT(year from FL_DATE) AS Year FROM airlinedelay GROUP BY ORIGIN, Year ORDER BY AverageDelayPerCarrier DESC")

averageDelayPerDepartureLocationEachYear.show(50)

+------+----------------------+----+
|ORIGIN|AverageDelayPerCarrier|Year|
+------+----------------------+----+
|   PPG|    201.55555555555554|2016|
|   ENV|                 157.0|2016|
|   DVL|     122.1344537815126|2018|
|   PPG|     99.58064516129032|2018|
|   GCK|     99.08108108108108|2016|
|   JMS|     97.18536585365854|2018|
|   CMX|     96.56744186046511|2018|
|   UST|     95.85714285714286|2017|
|   GCK|     92.89655172413794|2017|
|   RHI|     92.57462686567165|2017|
|   SLN|     88.72018348623853|2018|
|   LWB|     87.99009900990099|2018|
|   LCH|     86.76878612716763|2017|
|   MQT|     84.84939759036145|2016|
|   SAF|     82.25088339222614|2018|
|   SHD|     82.14367816091954|2018|
|   MEI|     81.23869346733669|2017|
|   RHI|     80.50381679389314|2018|
|   FSM|      80.1043956043956|2017|
|   HYS|     80.08812260536398|2018|
|   ITH|     79.99586776859505|2018|
|   MHK|     79.28909952606635|2017|
|   BGR|     79.12786885245902|2017|
|   EGE|       78.440329218107|2018|
|

DEPARTURE LOCATION WITH THE LARGEST AVERAGE DELAY

In [ ]:
spark.sql("SELECT ORIGIN, AVG(IF(DEP_DELAY >= 0 , DEP_DELAY, NULL)) AS AverageDelay,EXTRACT(year from FL_DATE) AS Year FROM airlinedelay GROUP BY ORIGIN, Year ORDER BY AverageDelay DESC").show()

+------+------------------+----+
|ORIGIN|      AverageDelay|Year|
+------+------------------+----+
|   PPG|201.55555555555554|2016|
|   ENV|             157.0|2016|
|   DVL| 122.1344537815126|2018|
|   PPG| 99.58064516129032|2018|
|   GCK| 99.08108108108108|2016|
|   JMS| 97.18536585365854|2018|
|   CMX| 96.56744186046511|2018|
|   UST| 95.85714285714286|2017|
|   GCK| 92.89655172413794|2017|
|   RHI| 92.57462686567165|2017|
|   SLN| 88.72018348623853|2018|
|   LWB| 87.99009900990099|2018|
|   LCH| 86.76878612716763|2017|
|   MQT| 84.84939759036145|2016|
|   SAF| 82.25088339222614|2018|
|   SHD| 82.14367816091954|2018|
|   MEI| 81.23869346733669|2017|
|   RHI| 80.50381679389314|2018|
|   FSM|  80.1043956043956|2017|
|   HYS| 80.08812260536398|2018|
+------+------------------+----+
only showing top 20 rows



In [ ]:
spark.sql("SELECT a1.ORIGIN, AVG(IF(a1.DEP_DELAY >= 0 , a1.DEP_DELAY, NULL)) AS AverageDelay,EXTRACT(year from a1.FL_DATE) AS Year \
FROM airlinedelay a1 WHERE AverageDelay IN ( SELECT a2.ORIGIN, AVG(IF(a2.DEP_DELAY >= 0 , a2.DEP_DELAY, NULL)) AS AverageDelay1,EXTRACT(year from a2.FL_DATE) AS Year1 \
FROM airlinedelay a2 WHERE a1.Year=a2.Year1 GROUP BY a2.ORIGIN ORDER BY AverageDelay1 DESC LIMIT 1) GROUP BY a1.ORIGIN, Year \
 ")

ParseException: 
Syntax error at or near 'WHERE'(line 1, pos 166)

== SQL ==
SELECT a1.ORIGIN, AVG(IF(a1.DEP_DELAY >= 0 , a1.DEP_DELAY, NULL)) AS AverageDelay,EXTRACT(year from a1.FL_DATE) AS Year FROM airlinedelay a1 GROUP BY a1.ORIGIN, Year WHERE AverageDelay IN ( SELECT a2.ORIGIN, AVG(IF(a2.DEP_DELAY >= 0 , a2.DEP_DELAY, NULL)) AS AverageDelay1,EXTRACT(year from a2.FL_DATE) AS Year1 FROM airlinedelay a2 WHERE a1.Year=a2.Year1 GROUP BY a2.ORIGIN ORDER BY AverageDelay1 DESC LIMIT 1) 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------^^^
